In [1]:
"""
MongoDB Database Connection and Operations Module

This module provides functions to connect to a MongoDB database
and perform various data extraction operations.
"""

import pymongo
from pymongo import MongoClient
import pandas as pd
import json
from datetime import datetime
import os
from urllib.parse import quote_plus

# Database connection constants
USER_NAME = "alfred"
PASS = "alfred-coco-cola"
MONGO_CLIENT = 'alfred-coco-cola'
MONGO_COLLECTION = 'email_threads'
MONGO_HOST = '172.178.91.142'
MONGO_PORT = 27017



/var/folders/4s/gb51cxx975xgdgnzpf1r7mr40000gp/T/ipykernel_2466/2904192211.py:10: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
import pymongo
import pandas as pd
from pymongo import MongoClient
import json
from bson import json_util

# Connect to the MongoDB database
# Assuming the connection details are the same as in the app.py file
client = MongoClient('localhost', 27017)  # Default MongoDB connection
db = client['alfred-coco-cola']  # Use the database name from the codebase
collection = db['email_threads']  # Access the email_threads collection

# Fetch all documents from the collection
email_threads = list(collection.find())

# Print the count of documents
print(f"Total documents in email_threads collection: {len(email_threads)}")

# Convert to a more readable format
# This handles MongoDB-specific types like ObjectId
def parse_json(data):
    return json.loads(json_util.dumps(data))

# Display the first document to see its structure
if email_threads:
    print("\nSample document structure:")
    print(json.dumps(parse_json(email_threads[0]), indent=2))

# Convert the data to a pandas DataFrame for easier analysis
# First flatten the documents to handle nested structures
flat_data = [parse_json(doc) for doc in email_threads]
df = pd.json_normalize(flat_data)

# Display the DataFrame
print("\nDataFrame view of email_threads:")
display(df)

# If you want to see all documents in detail
print("\nAll documents in email_threads:")
for i, thread in enumerate(email_threads):
    print(f"\n--- Document {i+1} ---")
    print(json.dumps(parse_json(thread), indent=2))

ServerSelectionTimeoutError: localhost:27017: [Errno 61] Connection refused, Timeout: 30s, Topology Description: <TopologyDescription id: 680774f04fbaa6c633a27322, topology_type: Unknown, servers: [<ServerDescription ('localhost', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('localhost:27017: [Errno 61] Connection refused')>]>

In [4]:
from app import load_chat_data

Successfully connected to MongoDB: alfred-coco-cola
Successfully connected to MongoDB: alfred-coco-cola


In [5]:
chat_data = load_chat_data()

Fetching chat data directly from MongoDB...
Processed 2 users with 5 sessions
Successfully loaded data for 2 users from MongoDB


In [6]:
chat_data

{'LT_samp_demo': {'session_8ccc9bc8ad': {'chat_history': [],
   'projects': [{'projectID': 'project_solar_panel_electrical_&_structural_setup',
     'projectName': 'Solar Panel Electrical & Structural Setup',
     'tasks': [{'Task': 'Mount Solar Panels on Racks',
       'TaskID': 'task_75459e217700493bbe55f86615f5ffc0',
       'AssignedTo': 'Aryan',
       'Deadline': '07-05-2025',
       'Status': 'in_progress',
       'last_updated': '2025-05-05T11:06:24.127761',
       'Comment': 'None',
       'Dependencies': ['None'],
       'email_thread_chain': [],
       'projectID': 'project_solar_panel_electrical_&_structural_setup',
       'ai_insight_generated': True},
      {'Task': 'String Cabling for Panels',
       'TaskID': 'task_9668c114d0c6406aa1d1ac823f18b0bf',
       'AssignedTo': 'Satya',
       'Deadline': '01-08-2025',
       'Status': 'Blocked',
       'last_updated': '2025-05-05T10:27:10.656740',
       'Comment': 'Cabling started; minor alignment issue found',
       'Depende

In [7]:
if "LT_samp_demo" not in chat_data:
        print(f"User not found: LT_samp_demo")

In [8]:
session_list = []
    
    # Get all sessions for this user
for session_id in chat_data["LT_samp_demo"].keys():
        # session_data = chat_data["LT_samp_demo"][session_id]
        print(session_id)

session_8ccc9bc8ad
session_de901082c7
session_b9ea182490
ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4
1db14a48


In [2]:
def get_user_sessions(user_id):
    chat_data = load_chat_data()
    if not chat_data:
        return ([]), 200

    # Check if user exists in our data
    if user_id not in chat_data:
        print(f"User not found: {user_id}")
        return ([]), 200
    
    print(f"Found user: {user_id}")
    session_list = []
    
    # Get all sessions for this user
    for session_id in chat_data[user_id].keys():
        session_data = chat_data[user_id][session_id]
        
        # Check if this session uses the new schema format
        if 'chat_history' not in session_data:
            print(f"Skipping session {session_id} - old schema format")
            continue
            
        messages = session_data.get('chat_history', [])
        
        # Calculate message count and timestamps
        message_count = len(messages)
        created_at = "Unknown"
        last_activity = "Unknown"
        
        if message_count > 0:
            # Try to get timestamps
            if messages[0].get('timestamp'):
                timestamp = messages[0].get('timestamp')
                if isinstance(timestamp, dict) and '$date' in timestamp:
                    created_at = timestamp['$date']
                else:
                    created_at = str(timestamp)
            
            if messages[-1].get('timestamp'):
                timestamp = messages[-1].get('timestamp')
                if isinstance(timestamp, dict) and '$date' in timestamp:
                    last_activity = timestamp['$date']
                else:
                    last_activity = str(timestamp)
        
        session_list.append({
            "id": session_id,
            "messageCount": message_count,
            "createdAt": created_at,
            "lastActivity": last_activity
        })
        print(f"Found session with new schema: {session_id} with {message_count} messages")
    
    return (session_list)

In [3]:
x = get_user_sessions("LT_samp_demo")

Fetching chat data directly from MongoDB...
Processed 2 users with 5 sessions
Successfully loaded data for 2 users from MongoDB
Found user: LT_samp_demo
Found session with new schema: session_8ccc9bc8ad with 0 messages
Found session with new schema: session_de901082c7 with 0 messages
Found session with new schema: session_b9ea182490 with 0 messages
Found session with new schema: ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4 with 3 messages
Found session with new schema: 1db14a48 with 1 messages


In [4]:
x

[{'id': 'session_8ccc9bc8ad',
  'messageCount': 0,
  'createdAt': 'Unknown',
  'lastActivity': 'Unknown'},
 {'id': 'session_de901082c7',
  'messageCount': 0,
  'createdAt': 'Unknown',
  'lastActivity': 'Unknown'},
 {'id': 'session_b9ea182490',
  'messageCount': 0,
  'createdAt': 'Unknown',
  'lastActivity': 'Unknown'},
 {'id': 'ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4',
  'messageCount': 3,
  'createdAt': '2025-05-07 14:56:22.046000',
  'lastActivity': '2025-05-07 11:14:29.653000'},
 {'id': '1db14a48',
  'messageCount': 1,
  'createdAt': '2025-05-07 11:40:53.027000',
  'lastActivity': '2025-05-07 11:40:53.027000'}]

In [2]:
from db import connect_to_mongodb, extract_chat_histories, save_to_json, MONGO_CLIENT, MONGO_COLLECTION

db_client, _ = connect_to_mongodb()

db = db_client[MONGO_CLIENT]

Successfully connected to MongoDB: alfred-coco-cola


In [ ]:
("LT_samp_demo", "ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4")

In [5]:
chat_data = load_chat_data()
session_data = chat_data["LT_samp_demo"]["ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4"]

Fetching chat data directly from MongoDB...
Processed 2 users with 5 sessions
Successfully loaded data for 2 users from MongoDB


In [6]:
chat_data

{'LT_samp_demo': {'session_8ccc9bc8ad': {'chat_history': [],
   'projects': [{'projectID': 'project_solar_panel_electrical_&_structural_setup',
     'projectName': 'Solar Panel Electrical & Structural Setup',
     'tasks': [{'Task': 'Mount Solar Panels on Racks',
       'TaskID': 'task_75459e217700493bbe55f86615f5ffc0',
       'AssignedTo': 'Aryan',
       'Deadline': '07-05-2025',
       'Status': 'in_progress',
       'last_updated': '2025-05-05T11:06:24.127761',
       'Comment': 'None',
       'Dependencies': ['None'],
       'email_thread_chain': [],
       'projectID': 'project_solar_panel_electrical_&_structural_setup',
       'ai_insight_generated': True},
      {'Task': 'String Cabling for Panels',
       'TaskID': 'task_9668c114d0c6406aa1d1ac823f18b0bf',
       'AssignedTo': 'Satya',
       'Deadline': '01-08-2025',
       'Status': 'Blocked',
       'last_updated': '2025-05-05T10:27:10.656740',
       'Comment': 'Cabling started; minor alignment issue found',
       'Depende

In [7]:
session_data

{'chat_history': [{'message_id': '7f86d395-73bc-4439-bc9d-64987129094d',
   'timestamp': datetime.datetime(2025, 5, 7, 14, 56, 22, 46000),
   'messages': [{'role': 'user',
     'content': 'what are the tasks of Solar Panel Electrical & Structural Setup'},
    {'role': 'function',
     'name': 'fetch_project_or_task_details',
     'content': 'function_response: function_response: query: {\'collection\': \'email_threads\', \'pipeline\': [{\'$unwind\': \'$sessions\'}, {\'$unwind\': \'$sessions.projects\'}, {\'$unwind\': \'$sessions.projects.tasks\'}, {\'$match\': {\'userid\': \'LT_samp_demo\', \'sessions.projects.projectName\': {\'$regex\': \'Solar Panel Electrical & Structural Setup\', \'$options\': \'i\'}}}, {\'$project\': {\'_id\': 0, \'userid\': 1, \'user_email\': 1, \'session_id\': \'$sessions.session_id\', \'last_updated_at\': \'$sessions.last_updated_at\', \'projectID\': \'$sessions.projects.projectID\', \'projectName\': \'$sessions.projects.projectName\', \'task\': \'$sessions.pro

In [9]:
aw_msgs = session_data.get('chat_history', [])
projects = session_data.get('projects', [])
tasks = session_data.get('tasks', [])
email_thread_chain = session_data.get('email_thread_chain', [])
email_thread_id = session_data.get('email_thread_id', None)
fb_coll = db['alfred_feedback']

In [10]:
aw_msgs

[{'message_id': '7f86d395-73bc-4439-bc9d-64987129094d',
  'timestamp': datetime.datetime(2025, 5, 7, 14, 56, 22, 46000),
  'messages': [{'role': 'user',
    'content': 'what are the tasks of Solar Panel Electrical & Structural Setup'},
   {'role': 'function',
    'name': 'fetch_project_or_task_details',
    'content': 'function_response: function_response: query: {\'collection\': \'email_threads\', \'pipeline\': [{\'$unwind\': \'$sessions\'}, {\'$unwind\': \'$sessions.projects\'}, {\'$unwind\': \'$sessions.projects.tasks\'}, {\'$match\': {\'userid\': \'LT_samp_demo\', \'sessions.projects.projectName\': {\'$regex\': \'Solar Panel Electrical & Structural Setup\', \'$options\': \'i\'}}}, {\'$project\': {\'_id\': 0, \'userid\': 1, \'user_email\': 1, \'session_id\': \'$sessions.session_id\', \'last_updated_at\': \'$sessions.last_updated_at\', \'projectID\': \'$sessions.projects.projectID\', \'projectName\': \'$sessions.projects.projectName\', \'task\': \'$sessions.projects.tasks\'}}]}\nresu

In [12]:
msg_ids = [m.get('message_id') for m in aw_msgs]


In [11]:
for m in aw_msgs:
    print(m.get('message_id'))

7f86d395-73bc-4439-bc9d-64987129094d
fbf41bcd-7691-4a18-9b65-11f324d56b00
c1bf5361-b097-4842-b46d-ab5a92179792


In [13]:
msg_ids

['7f86d395-73bc-4439-bc9d-64987129094d',
 'fbf41bcd-7691-4a18-9b65-11f324d56b00',
 'c1bf5361-b097-4842-b46d-ab5a92179792']

In [15]:
fb_docs = list(fb_coll.find({'message_id': {'$in': msg_ids}}))

In [16]:
fb_docs

[]

In [17]:
fb_map = {doc['message_id']: doc for doc in fb_docs}

In [18]:
fb_map

{}

In [21]:
aw_msgs


[{'message_id': '7f86d395-73bc-4439-bc9d-64987129094d',
  'timestamp': datetime.datetime(2025, 5, 7, 14, 56, 22, 46000),
  'messages': [{'role': 'user',
    'content': 'what are the tasks of Solar Panel Electrical & Structural Setup'},
   {'role': 'function',
    'name': 'fetch_project_or_task_details',
    'content': 'function_response: function_response: query: {\'collection\': \'email_threads\', \'pipeline\': [{\'$unwind\': \'$sessions\'}, {\'$unwind\': \'$sessions.projects\'}, {\'$unwind\': \'$sessions.projects.tasks\'}, {\'$match\': {\'userid\': \'LT_samp_demo\', \'sessions.projects.projectName\': {\'$regex\': \'Solar Panel Electrical & Structural Setup\', \'$options\': \'i\'}}}, {\'$project\': {\'_id\': 0, \'userid\': 1, \'user_email\': 1, \'session_id\': \'$sessions.session_id\', \'last_updated_at\': \'$sessions.last_updated_at\', \'projectID\': \'$sessions.projects.projectID\', \'projectName\': \'$sessions.projects.projectName\', \'task\': \'$sessions.projects.tasks\'}}]}\nresu

In [23]:
merged_msgs = []

"""'id': msg_id,
            'role': 
            'content': m.get('content', ''),
            'timestamp': ts,
            'feedback': doc_fb.get('feedback'),
            'comments': doc_fb.get('comments', []),
            'sequence': m.get('sequence', i)
            """

for m in aw_msgs:
    msg_id = m.get('message_id')
    ts = m.get('timestamp', 'Unknown time')
    if isinstance(ts, dict) and 'date' in ts:
        ts = ts['date']
    # Normalize timestamp to ISO string if datetime object
    if hasattr(ts, 'isoformat'):
        ts = ts.isoformat()

    for role in m.get('messages'):
        role_data = role.get('role')
        content = role.get('content')
        msg_data = {
            'id': msg_id,
            'role': role_data,
            'content': content,
            'timestamp': ts,
            'sequence': m.get('sequence', i)
        }
        merged_msgs.append(msg_data)

In [24]:
merged_msgs

[{'id': '7f86d395-73bc-4439-bc9d-64987129094d',
  'role': 'user',
  'content': 'what are the tasks of Solar Panel Electrical & Structural Setup',
  'timestamp': '2025-05-07T14:56:22.046000',
  'sequence': 0},
 {'id': '7f86d395-73bc-4439-bc9d-64987129094d',
  'role': 'function',
  'content': 'function_response: function_response: query: {\'collection\': \'email_threads\', \'pipeline\': [{\'$unwind\': \'$sessions\'}, {\'$unwind\': \'$sessions.projects\'}, {\'$unwind\': \'$sessions.projects.tasks\'}, {\'$match\': {\'userid\': \'LT_samp_demo\', \'sessions.projects.projectName\': {\'$regex\': \'Solar Panel Electrical & Structural Setup\', \'$options\': \'i\'}}}, {\'$project\': {\'_id\': 0, \'userid\': 1, \'user_email\': 1, \'session_id\': \'$sessions.session_id\', \'last_updated_at\': \'$sessions.last_updated_at\', \'projectID\': \'$sessions.projects.projectID\', \'projectName\': \'$sessions.projects.projectName\', \'task\': \'$sessions.projects.tasks\'}}]}\nresult: [\'{"userid": "LT_samp_d

In [ ]:
def get_interactions():
    chat_data = load_chat_data()
    if not chat_data:
        print("No chat data available, returning empty list")
        return ([]), 200
    
    interactions = []
    try:
        for user_id, sessions in chat_data.items():
            for session_id, session_data in sessions.items():
                # Check if session uses the new schema format
                if 'chat_history' not in session_data:
                    continue
                    
                chat_history = session_data['chat_history']
                
                if not chat_history or not isinstance(chat_history, list):
                    continue
                
                print(f"Processing session {session_id} with {len(chat_history)} chat history items")
                
                # Process each message in the chat history
                for message in chat_history:
                    # Validate message structure
                    if not isinstance(message, dict) or 'messages' not in message:
                        continue
                    
                    message_id = message.get('message_id')
                    timestamp = message.get('timestamp')
                    messages_list = message.get('messages', [])
                    
                    # Ensure timestamp is a string
                    if isinstance(timestamp, dict) and '$date' in timestamp:
                        timestamp = timestamp['$date']
                    elif hasattr(timestamp, 'isoformat'):  # Python datetime object
                        timestamp = timestamp.isoformat()
                    
                    # Find user and assistant messages
                    user_content = None
                    assistant_content = None
                    
                    for msg in messages_list:
                        if msg.get('role') == 'user':
                            user_content = msg.get('content')
                        elif msg.get('role') == 'assistant':
                            assistant_content = msg.get('content')
                    
                    # Only add to interactions if we have both user and assistant content
                    if user_content and assistant_content:
                        interactions.append({
                            'id': message_id,  # Keep using 'id' for frontend compatibility
                            'userPrompt': user_content,
                            'aiResponse': assistant_content,
                            'timestamp': str(timestamp),
                            'agents': [],
                            'rating': None,
                            'comments': [],
                            'user': {
                                'name': user_id,
                                'avatar': ''
                            }
                        })
    
    except Exception as e:
        print(f"Error formatting interactions: {e}")
        import traceback
        traceback.print_exc()
    
    # Merge stored feedback/comments from DB
    try:
        fb_coll = db['alfred_feedback']
        
        # Create a list of message IDs to query
        msg_ids = [item['id'] for item in interactions if item['id']]
        
        if msg_ids:
            # Try looking up feedback by both _id and message_id fields
            fb_docs_by_id = list(fb_coll.find({'_id': {'$in': msg_ids}}))
            fb_docs_by_msg_id = list(fb_coll.find({'message_id': {'$in': msg_ids}}))
            
            # Combine the results, prioritizing _id matches
            fb_map = {doc['_id']: doc for doc in fb_docs_by_id}
            
            # Add message_id matches only if _id match doesn't exist
            for doc in fb_docs_by_msg_id:
                msg_id = doc['message_id']
                if msg_id not in fb_map:
                    fb_map[msg_id] = doc
            
            # Update interactions with feedback data
            for item in interactions:
                if not item['id']:
                    continue
                    
                doc = fb_map.get(item['id'], {})
                # override defaults if present
                item['rating'] = doc.get('feedback', item.get('rating'))
                item['comments'] = doc.get('comments', item.get('comments', []))
                
                print(f"Message {item['id']}: rating={item['rating']}, comments={item['comments']}")
    except Exception as e:
        print(f"Error merging feedback: {e}")
        import traceback
        traceback.print_exc()
    
    print(f"Returning {len(interactions)} interactions with persisted feedback")
    return (interactions)

In [19]:
merged_msgs = []
for i, m in enumerate(aw_msgs):
        msg_id = m.get('message_id')
        ts = m.get('timestamp', 'Unknown time')
        if isinstance(ts, dict) and 'date' in ts:
            ts = ts['date']
        # Normalize timestamp to ISO string if datetime object
        if hasattr(ts, 'isoformat'):
            ts = ts.isoformat()
        
        # Get feedback doc or empty dict if not found
        doc_fb = fb_map.get(msg_id, {})
        for role in m.get('messages'):
            role_data = role.get('role')
            content = role.get('content')
        # Create merged message with all data
        msg_data = {
            'id': msg_id,
            'role': 
            'content': m.get('content', ''),
            'timestamp': ts,
            'feedback': doc_fb.get('feedback'),
            'comments': doc_fb.get('comments', []),
            'sequence': m.get('sequence', i)
        }
        
        merged_msgs.append(msg_data)

In [20]:
merged_msgs

[{'id': '7f86d395-73bc-4439-bc9d-64987129094d',
  'role': 'unknown',
  'content': '',
  'timestamp': '2025-05-07T14:56:22.046000',
  'feedback': None,
  'comments': [],
  'sequence': 0},
 {'id': 'fbf41bcd-7691-4a18-9b65-11f324d56b00',
  'role': 'unknown',
  'content': '',
  'timestamp': '2025-05-07T09:32:20.831000',
  'feedback': None,
  'comments': [],
  'sequence': 1},
 {'id': 'c1bf5361-b097-4842-b46d-ab5a92179792',
  'role': 'unknown',
  'content': '',
  'timestamp': '2025-05-07T11:14:29.653000',
  'feedback': None,
  'comments': [],
  'sequence': 2}]

In [1]:
def get_session_chat(user_id, session_id):
    chat_data = load_chat_data()
    if not chat_data:
        return ([]), 200
    
    # Check if user exists
    if user_id not in chat_data:
        return ([]), 200
    
    # Check if session exists for this user
    if session_id not in chat_data[user_id]:
        return ([]), 200
    
    # Merge with feedback DB
    session_data = chat_data[user_id][session_id]
    raw_msgs = session_data.get('chat_history', [])
    projects = session_data.get('projects', [])
    tasks = session_data.get('tasks', [])
    email_thread_chain = session_data.get('email_thread_chain', [])
    email_thread_id = session_data.get('email_thread_id', None)
    fb_coll = db['alfred_feedback']
    
    # Build or extract message_ids
    msg_ids = [m.get('message_id') for i, m in enumerate(raw_msgs)]
    
    # Instead of creating placeholder documents for ALL messages,
    # just fetch existing feedback documents and leave creation to happen only when needed
    fb_docs = list(fb_coll.find({'message_id': {'$in': msg_ids}}))
    
    # Map feedback docs by message_id
    fb_map = {doc['message_id']: doc for doc in fb_docs}
    
    # Merge messages with feedback
    merged_msgs = []
    for i, m in enumerate(raw_msgs):
        msg_id = m.get('message_id') or msg_ids[i]
        ts = m.get('timestamp', 'Unknown time')
        if isinstance(ts, dict) and 'date' in ts:
            ts = ts['date']
        # Normalize timestamp to ISO string if datetime object
        if hasattr(ts, 'isoformat'):
            ts = ts.isoformat()
        
        # Get feedback doc or empty dict if not found
        doc_fb = fb_map.get(msg_id, {})
        
        # Create merged message with all data
        msg_data = {
            'id': msg_id,
            'role': m.get('role', 'unknown'),
            'content': m.get('content', ''),
            'timestamp': ts,
            'feedback': doc_fb.get('feedback'),
            'comments': doc_fb.get('comments', []),
            'sequence': m.get('sequence', i)
        }
        
        merged_msgs.append(msg_data)
    
    # Sort messages by sequence number first (most reliable), then by timestamp as fallback
    try:
        # First try to sort by sequence number which is most reliable
        merged_msgs.sort(key=lambda x: x.get('sequence', float('inf')))
    except Exception as e:
        print(f"Error sorting messages by sequence: {e}")
        # Fallback: try to sort by timestamp
        try:
            merged_msgs.sort(key=lambda x: x.get('timestamp', ''))
        except Exception as e:
            print(f"Error sorting messages by timestamp: {e}")

    # Return all structured session data to frontend
    return ({
        'messages': merged_msgs,
        'projects': projects,
        'tasks': tasks,
        'email_thread_chain': email_thread_chain,
        'email_thread_id': email_thread_id
    })

In [20]:
y=get_session_chat("LT_samp_demo", "ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4")

Fetching chat data directly from MongoDB...
Processed 2 users with 5 sessions
Successfully loaded data for 2 users from MongoDB


In [21]:
y

{'messages': [{'id': 'LT_samp_demo_ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4_0',
   'role': 'unknown',
   'content': '',
   'timestamp': '2025-05-07T14:56:22.046000',
   'feedback': None,
   'comments': [],
   'sequence': 0},
  {'id': 'LT_samp_demo_ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4_1',
   'role': 'unknown',
   'content': '',
   'timestamp': '2025-05-07T09:32:20.831000',
   'feedback': None,
   'comments': [],
   'sequence': 1},
  {'id': 'LT_samp_demo_ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4_2',
   'role': 'unknown',
   'content': '',
   'timestamp': '2025-05-07T11:14:29.653000',
   'feedback': None,
   'comments': [],
   'sequence': 2}],
 'projects': [],
 'tasks': [],
 'email_thread_chain': [],
 'email_thread_id': None}

In [91]:
chat_data = load_chat_data()

Fetching chat data directly from MongoDB...
Processed 2 users with 8 sessions
Successfully loaded data for 2 users from MongoDB


In [93]:
chat_data.items()

dict_items([('LT_samp_demo', {'session_8ccc9bc8ad': {'chat_history': [], 'projects': [{'projectID': 'project_solar_panel_electrical_&_structural_setup', 'projectName': 'Solar Panel Electrical & Structural Setup', 'tasks': [{'Task': 'Mount Solar Panels on Racks', 'TaskID': 'task_75459e217700493bbe55f86615f5ffc0', 'AssignedTo': 'Aryan', 'Deadline': '07-05-2025', 'Status': 'in_progress', 'last_updated': '2025-05-05T11:06:24.127761', 'Comment': 'None', 'Dependencies': ['None'], 'email_thread_chain': [], 'projectID': 'project_solar_panel_electrical_&_structural_setup', 'ai_insight_generated': True}, {'Task': 'String Cabling for Panels', 'TaskID': 'task_9668c114d0c6406aa1d1ac823f18b0bf', 'AssignedTo': 'Satya', 'Deadline': '01-08-2025', 'Status': 'completed', 'last_updated': '2025-05-05T10:27:10.656740', 'Comment': 'Cabling started; minor alignment issue found', 'Dependencies': ['None'], 'email_thread_chain': [], 'projectID': 'project_solar_panel_electrical_&_structural_setup', 'ai_insight_ge

In [100]:
chat_data
for user_id, sessions in chat_data.items():
    for session_id, session_data in sessions.items():
        # Check if session uses the new schema format
        if 'chat_history' not in session_data:
            continue
            
        chat_history = session_data['chat_history']
        
        if not chat_history or not isinstance(chat_history, list):
            continue
        
        # Process each message in the chat history
        for message in chat_history:
            # Validate message structure
            if not isinstance(message, dict) or 'messages' not in message:
                continue

            message_id = message.get('message_id')
            timestamp = message.get('timestamp')
            messages_list = message.get('messages', [])
            
            # Ensure timestamp is a string
            if isinstance(timestamp, dict) and '$date' in timestamp:
                timestamp = timestamp['$date']
            elif hasattr(timestamp, 'isoformat'):# Python datetime object
                timestamp = timestamp.isoformat()

{'message_id': '7f86d395-73bc-4439-bc9d-64987129094d', 'timestamp': datetime.datetime(2025, 5, 7, 14, 56, 22, 46000), 'messages': [{'role': 'user', 'content': 'what are the tasks of Solar Panel Electrical & Structural Setup'}, {'role': 'function', 'name': 'fetch_project_or_task_details', 'content': 'function_response: function_response: query: {\'collection\': \'email_threads\', \'pipeline\': [{\'$unwind\': \'$sessions\'}, {\'$unwind\': \'$sessions.projects\'}, {\'$unwind\': \'$sessions.projects.tasks\'}, {\'$match\': {\'userid\': \'LT_samp_demo\', \'sessions.projects.projectName\': {\'$regex\': \'Solar Panel Electrical & Structural Setup\', \'$options\': \'i\'}}}, {\'$project\': {\'_id\': 0, \'userid\': 1, \'user_email\': 1, \'session_id\': \'$sessions.session_id\', \'last_updated_at\': \'$sessions.last_updated_at\', \'projectID\': \'$sessions.projects.projectID\', \'projectName\': \'$sessions.projects.projectName\', \'task\': \'$sessions.projects.tasks\'}}]}\nresult: [\'{"userid": "L

In [108]:
for user_id, sessions in chat_data.items():
    for session_id, session_data in sessions.items():
        # Check if session uses the new schema format
        if 'chat_history' not in session_data:
            continue
        
        chat_history = session_data['chat_history']
        
        if not chat_history or not isinstance(chat_history, list):
            continue
        
        # Process each message in the chat history
        for message in chat_history:
            # Validate message structure
            if not isinstance(message, dict) or 'messages' not in message:
                continue

            print(message)

In [106]:
messages_list = message.get('messages', [])


In [107]:
messages_list

[{'role': 'user', 'content': 'hi'},
 {'role': 'assistant', 'content': 'Hello! How can I assist you today?'}]

In [88]:
def get_interactions():
    chat_data = load_chat_data()
    if not chat_data:
        print("No chat data available, returning empty list")
        return ([]), 200
    
    interactions = []
    
    for user_id, sessions in chat_data.items():
        for session_id, session_data in sessions.items():
            # Check if session uses the new schema format
            if 'chat_history' not in session_data:
                continue
                
            chat_history = session_data['chat_history']
            
            if not chat_history or not isinstance(chat_history, list):
                continue
            
            print(f"Processing session {session_id} with {len(chat_history)} chat history items")
            
            # Process each message in the chat history
            for message in chat_history:
                # Validate message structure
                if not isinstance(message, dict) or 'messages' not in message:
                    continue
                
                

                message_id = message.get('message_id')
                timestamp = message.get('timestamp')
                messages_list = message.get('messages', [])
                
                # Ensure timestamp is a string
                if isinstance(timestamp, dict) and '$date' in timestamp:
                    timestamp = timestamp['$date']
                elif hasattr(timestamp, 'isoformat'):  # Python datetime object
                    timestamp = timestamp.isoformat()
                
                # Find user and assistant messages
                user_content = None
                assistant_content = None
                function_name = None
                function_response = None
                
                for msg in messages_list:
                    if msg.get('role') == 'user':
                        user_content = msg.get('content')
                    elif msg.get('role') == 'assistant':
                        assistant_content = msg.get('content')
                    elif msg.get('role') == 'function':
                        function_name = msg.get('name')
                        function_response = msg.get('content')
                
                # Only add to interactions if we have both user and assistant content
                if user_content and assistant_content:
                    interactions.append({
                        'id': message_id,  # Keep using 'id' for frontend compatibility
                        'userPrompt': user_content,
                        'aiResponse': assistant_content,
                        'timestamp': str(timestamp),
                        'function_name': function_name,
                        'function_response': function_response,
                        'agents': [],
                        'rating': None,
                        'comments': [],
                        'user': {
                            'name': user_id,
                            'avatar': ''
                        }
                    })



# Merge stored feedback/comments from DB

    fb_coll = db['alfred_feedback']
    
    # Create a list of message IDs to query
    msg_ids = [item['id'] for item in interactions if item['id']]
    
    if msg_ids:
        # Try looking up feedback by both _id and message_id fields
        fb_docs_by_id = list(fb_coll.find({'_id': {'$in': msg_ids}}))
        fb_docs_by_msg_id = list(fb_coll.find({'message_id': {'$in': msg_ids}}))
        
        # Combine the results, prioritizing _id matches
        fb_map = {doc['_id']: doc for doc in fb_docs_by_id}
        
        # Add message_id matches only if _id match doesn't exist
        for doc in fb_docs_by_msg_id:
            msg_id = doc['message_id']
            if msg_id not in fb_map:
                fb_map[msg_id] = doc
        
        # Update interactions with feedback data
        for item in interactions:
            if not item['id']:
                continue
                
            doc = fb_map.get(item['id'], {})
            # override defaults if present
            item['rating'] = doc.get('feedback', item.get('rating'))
            item['comments'] = doc.get('comments', item.get('comments', []))
            
            print(f"Message {item['id']}: rating={item['rating']}, comments={item['comments']}")
    
    
    print(f"Returning {len(interactions)} interactions with persisted feedback")
    return (interactions)

In [89]:
x = get_interactions()

Fetching chat data directly from MongoDB...
Processed 2 users with 8 sessions
Successfully loaded data for 2 users from MongoDB
Processing session ba56cd45-53ac-4e95-b8b3-2109a8bfc7f4 with 6 chat history items
Processing session 2f96e56e-0d6d-4ff3-814f-b3156fffd8c2 with 1 chat history items
Processing session bbed6634-1a91-4911-b327-3042b0cc667b with 2 chat history items
Processing session 4249af00-9ff6-4e92-a341-47fbd5e92b8f with 1 chat history items
Processing session f65edc80-1269-4d2e-967f-072ce7d20cce with 1 chat history items
Message 7f86d395-73bc-4439-bc9d-64987129094d: rating=None, comments=[]
Message fbf41bcd-7691-4a18-9b65-11f324d56b00: rating=None, comments=[]
Message 6083241a-42f0-400e-8661-cce7f026bf6a: rating=None, comments=[]
Message 408d6798-b544-4df5-9c01-2b853c3beb59: rating=None, comments=[]
Message 20866c76-2da8-485a-94ef-1b8ba4d760c2: rating=None, comments=[]
Message 68bd497e-be7a-4cf2-9d27-f23d2e54d47a: rating=None, comments=[]
Message 80cc8bc2-0d66-4210-a089-6320

In [90]:
x

[{'id': '7f86d395-73bc-4439-bc9d-64987129094d',
  'userPrompt': 'what are the tasks of Solar Panel Electrical & Structural Setup',
  'aiResponse': "Here are the tasks for the **Solar Panel Electrical & Structural Setup** project:\n\n**Project Name: Solar Panel Electrical & Structural Setup**\n\n1. **Task:** Mount Solar Panels on Racks\n   - **Assigned To:** Aryan\n   - **Deadline:** 07-05-2025\n   - **Status:** In Progress\n   - **Comments:** None\n   - **Dependencies:** None\n   - **Email:** aryan.p@pathsetter.io\n\n2. **Task:** String Cabling for Panels\n   - **Assigned To:** Satya\n   - **Deadline:** 01-08-2025\n   - **Status:** Blocked\n   - **Comments:** Cabling started; minor alignment issue found\n   - **Dependencies:** None\n   - **Email:** satya.n@pathsetter.io\n\n3. **Task:** Install DC Combiner Boxes\n   - **Assigned To:** Satya\n   - **Deadline:** 07-05-2025\n   - **Status:** Blocked\n   - **Comments:** None\n   - **Dependencies:** None\n   - **Email:** satya.n@pathsetter.i

In [84]:
from db import MONGO_COLLECTION
current_total_interactions = 0
    # Get all chat data
collection = db[MONGO_COLLECTION]

# Add a direct count query first as a fallback
total_docs = collection.count_documents({})
if total_docs > 0:
    current_total_interactions = total_docs  # Use this as initial value

# Process each document for more accurate counting
cursor = collection.find({})



In [86]:
for doc in cursor:
    for session_data in doc['sessions']:
        print(session_data)
        if 'chat_history' in session_data:
            chat_history = session_data['chat_history']
            current_total_interactions += len(chat_history)

In [87]:
# Reset counter for detailed counting
interaction_count = 0

# Process each document
for doc in cursor:
    for session in doc.get('sessions', []):
        messages = session.get('chat_history', [])
        if not messages:
            continue
        print(messages)
        interaction_count += len(messages)

print(f"Total interactions in current period: {interaction_count}")

Total interactions in current period: 0


In [109]:
cursor = collection.find({})

In [110]:
cursor

In [115]:
total_messages = 0
    
    # Check if sessions array exists
    if not data.get('sessions') or not isinstance(data['sessions'], list):
        return 0
    
    # Iterate through each session
    for session in data['sessions']:
        # Check if this session has chat_history
        if session.get('chat_history') and isinstance(session['chat_history'], list):
            # For each chat_history entry, count the messages
            for history_item in session['chat_history']:
                if history_item.get('messages') and isinstance(history_item['messages'], list):
                    total_messages += len(history_item['messages'])

Total interactions in current period: 0
